Let's make this interface really simple for now and extensible for later. The main purpose of the interface is to make it easy to manually label our data. To this end, I will read in our JSON file continuously, record user information, and save it. There will be minimal error handling or edge case testing. We assume the user will select the proper results each time. User progress will not be saved. If overrides are needed, please override throught the interface code itself. This is be very jank, but should be fast, simple, and work for our very specific use case.

Make continuous annotation_ui file stream. We will assume that the user is keeping track of which annotations they've already made on files.

In [ ]:
import gradio as gr
import re
import json

In [ ]:
def get_speaker(opening_statement):
    return re.findall(r"<speaker>(.*?)</speaker>", opening_statement)[0]

def get_text(opening_statement):
    return re.findall(r"<text>(.*?)</text>", opening_statement)[0]

def format_opening_statement(opening_statement):
    return f"**{get_speaker(opening_statement)}** {get_text(opening_statement)}"

def get_year(transcript_id):
    return transcript_id[:4]

def get_docket(transcript_id):
    return transcript_id[5:11]

In [85]:
with open('datasets/2024_questions_for_eval.json') as f:
    all_data_to_annotate = json.load(f)

def generate_markdown(transcript_id, petitioner_or_respondent, justice_name):
    selected_transcript = [x for x in all_data_to_annotate if x["transcript_id"] == transcript_id][0]
    print(selected_transcript)
    opening_statement = selected_transcript[petitioner_or_respondent.lower()]["opening_statement"]

    return f"""
        # Docket {get_docket(transcript_id)}, {get_year(transcript_id)} 
        ## Opening Statement of {petitioner_or_respondent}
        {get_text(opening_statement)}
        ## {justice_name}'s Questions (from the Oral Argument Transcripts)
        """

with gr.Blocks() as annotation_ui:
    with gr.Column() as config_setup_row:
        # initial config set-up
        transcript_select_dropdown = gr.Dropdown((x["transcript_id"] for x in all_data_to_annotate), 
                                                label="Select Transcript to Annotate", interactive=True)
        petitioner_or_respondent_select = gr.Radio(["Petitioner", "Respondent"], label="Select Side", interactive=True)
        justice_select = gr.Radio(["Sonia Sotomayor", "Samuel Alito"], label="Select Justice", interactive=True, visible=False)
        question_select_button = gr.Button(value="Annotate!")

    with gr.Column() as listing:
        transcript_info = gr.Markdown()
    
    # config set-up interactive logic
    petitioner_or_respondent_select.select(fn=lambda: gr.Radio(visible=True), inputs=None, outputs=justice_select)
    justice_select.select(fn=lambda: gr.Button(visible=True), inputs=None, outputs=question_select_button)
    question_select_button.click(fn=lambda: gr.Row(visible=False), inputs=None, outputs=config_setup_row).then(
        generate_markdown, [transcript_select_dropdown, petitioner_or_respondent_select, justice_select], transcript_info
    )
annotation_ui.launch()


* Running on local URL:  http://127.0.0.1:7928

To create a public link, set `share=True` in `launch()`.


{'transcript_id': '2024.23-621-t01', 'petitioner': {'opening_statement': '<speaker>Erika L. Maley</speaker><text> Mr. Chief Justice, and may it please the Court:  The prevailing party is the party who wins the lawsuit, obtaining a final judgment in its favor, or at least a party who obtains a ruling that the defendant is liable on the merits of one or more claims, such as a summary judgment or a judgment as a matter of law. A preliminary injunction is neither a final judgment nor a determination that the defendant is liable on the merits for violating federal law. It is simply a threshold prediction of the likelihood of success based on a truncated record and an initial, often hasty assessment of the law that may well prove to be faulty as the case proceeds. It provid es no enduring relief. By its nature, it is a temporary procedural order that dissolves upon final judgment. A preliminary injunction, therefore, does not make a plaintiff a prevailing party  within the meaning of that le